In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as md
import os
import mne
import random
from sklearn.metrics import classification_report
from numpy import zeros, newaxis
from scipy import signal
from scipy import fftpack

In [104]:
MDD_dataset_Filenames = os.listdir('./full_dataset/MDD/EC/')
H_dataset_Filenames = os.listdir('./full_dataset/H/EC/')

MDD_dataset_list = []
H_dataset_list = []

for filename in MDD_dataset_Filenames:
    data = mne.io.read_raw_edf('./full_dataset/MDD/EC/' + str(filename))
    MDD_dataset_list.append(data)  
    
for filename in H_dataset_Filenames:
    data = mne.io.read_raw_edf('./full_dataset/H/EC/' + str(filename))
    H_dataset_list.append(data)  

Extracting EDF parameters from C:\ARK\python\MDD\full_dataset\MDD\EC\MDD S1 EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\ARK\python\MDD\full_dataset\MDD\EC\MDD S10 EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\ARK\python\MDD\full_dataset\MDD\EC\MDD S11  EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\ARK\python\MDD\full_dataset\MDD\EC\MDD S13 EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\ARK\python\MDD\full_dataset\MDD\EC\MDD S14 EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\ARK\python\MDD\full_dataset\MDD\EC\MDD S15 EC.edf...
EDF file detected
Setting channel info structure...
Creating ra

In [105]:
MDD_dataset_list[0].ch_names

['EEG Fp1-LE',
 'EEG F3-LE',
 'EEG C3-LE',
 'EEG P3-LE',
 'EEG O1-LE',
 'EEG F7-LE',
 'EEG T3-LE',
 'EEG T5-LE',
 'EEG Fz-LE',
 'EEG Fp2-LE',
 'EEG F4-LE',
 'EEG C4-LE',
 'EEG P4-LE',
 'EEG O2-LE',
 'EEG F8-LE',
 'EEG T4-LE',
 'EEG T6-LE',
 'EEG Cz-LE',
 'EEG Pz-LE',
 'EEG A2-A1']

In [113]:
MDD_dataframe = [] #MDD 환자의 데이터프레임 리스트
H_dataframe = [] #건강한 사람의 데이터프레임 리스트
y = []
for raw in MDD_dataset_list:
    raw_data = raw.get_data(return_times=True)
    channels = raw.ch_names
    T_data = np.transpose(raw_data[0])
    time = np.transpose(raw_data[1])
    temp = pd.DataFrame(T_data, columns = channels, index = time)
    MDD_dataframe.append(temp[['EEG Fp1-LE', 'EEG F3-LE', 'EEG F7-LE', 'EEG Fz-LE', 'EEG Fp2-LE', 'EEG F4-LE', 'EEG F8-LE']])
    y.append(1)
    
for raw in H_dataset_list:
    raw_data = raw.get_data(return_times=True)
    channels = raw.ch_names
    T_data = np.transpose(raw_data[0])
    time = np.transpose(raw_data[1])
    temp = pd.DataFrame(T_data, columns = channels, index = time)
    H_dataframe.append(temp[['EEG Fp1-LE', 'EEG F3-LE', 'EEG F7-LE', 'EEG Fz-LE', 'EEG Fp2-LE', 'EEG F4-LE', 'EEG F8-LE']])
    y.append(0)
y = np.asarray(y)    

In [114]:
MDD_fft_df = np.zeros((1, 7, 30000))
for df in MDD_dataframe:
    col = list(df.columns)
    fft = abs(fftpack.fft(np.array(df[col[0]])))[:30000]
    for i in range(1, len(col)):
        power = fftpack.fft(np.array(df[col[i]]))
        power = abs(power)
        fft = np.vstack([fft, power[:30000]])
    MDD_fft_df = np.vstack([MDD_fft_df[:,:,:], fft[newaxis,:,:]])
MDD_fft_df = MDD_fft_df[1:]

In [115]:
H_fft_df = np.zeros((1, 7, 30000))
for df in H_dataframe:
    col = list(df.columns)
    fft = abs(fftpack.fft(np.array(df[col[0]])))[:30000]
    for i in range(1, len(col)):
        power = fftpack.fft(np.array(df[col[i]]))
        power = abs(power)
        fft = np.vstack([fft, power[:30000]])
    H_fft_df = np.vstack([H_fft_df[:,:,:], fft[newaxis,:,:]])
H_fft_df = H_fft_df[1:]

In [153]:
all_fft = np.vstack([MDD_fft_df, H_fft_df])
print(all_fft.shape , y.shape)


features = all_fft.reshape(58,30000,7)
target = y
features.shape



(58, 7, 30000) (58,)


(58, 30000, 7)

In [159]:
from keras import models
from keras import layers
from keras import backend as K

accuracy_scores = []
prec_scores = []
rec_scores = []
f1_scores = []

from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=3)
for train, test in kfold.split(features, target):
    k_model = models.Sequential()

    k_model.add(layers.Dense(1, activation = 'sigmoid'))
    
    k_model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy', recall_m, precision_m, f1_m])
    
    k_model.fit(features[train], target[train], epochs=10, batch_size=512, verbose=0)
    
    scores = k_model.evaluate(features[test], target[test], verbose=0)
    
    print("%s: %.2f%%" % (k_model.metrics_names[1], scores[1]*100))
    print("%s: %.2f%%" % (k_model.metrics_names[2], scores[2]*100))
    print("%s: %.2f%%" % (k_model.metrics_names[3], scores[3]*100))
    print("%s: %.2f%%" % (k_model.metrics_names[4], scores[4]*100))
    print("\n")
    accuracy_scores.append(scores[1] * 100)
    prec_scores.append(scores[2])
    rec_scores.append(scores[3])
    f1_scores.append(scores[4])
    
print("%.2f (+/- %.2f%%)" % (np.mean(accuracy_scores), np.std(accuracy_scores)))
print("%.2f (+/- %.2f%%)" % (np.mean(prec_scores), np.std(prec_scores)))
print("%.2f (+/- %.2f%%)" % (np.mean(rec_scores), np.std(rec_scores)))
print("%.2f (+/- %.2f%%)" % (np.mean(f1_scores), np.std(f1_scores)))

IndexError: tuple index out of range

In [132]:
features[train][i]

array([[3.67218015e-03, 1.96008687e-04, 1.75297682e-04, ...,
        1.62042277e-05, 8.02909788e-06, 1.93463471e-05],
       [3.78808899e-03, 8.65287271e-05, 7.67966325e-05, ...,
        6.18513095e-06, 7.88748715e-06, 5.16121682e-06],
       [3.85229389e-03, 3.50912260e-05, 3.82679608e-05, ...,
        1.15896120e-05, 1.18638281e-05, 1.23796553e-06],
       ...,
       [3.71278325e-03, 1.31205885e-04, 1.45640466e-04, ...,
        1.55485331e-05, 1.86408613e-05, 1.30821007e-05],
       [3.73548498e-03, 9.41030022e-05, 1.09909791e-04, ...,
        6.92258111e-06, 8.70650702e-06, 8.54905618e-06],
       [3.74668583e-03, 1.15659392e-04, 1.10602677e-04, ...,
        2.50361293e-06, 1.30259420e-05, 4.51138760e-06]])

In [ ]:
plt.pcolormesh(t, f, np.abs(z))
plt.ylim(0,0.1)ff

In [ ]:
all_df = np.array(MDD_dataframe[0].T)[newaxis,:,:45000]

for i in range(1, len(MDD_dataframe)):
    b = np.array(MDD_dataframe[i].T)
    print(b.shape)
    print(all_df.shape)
    all_df = np.vstack([all_df, b[newaxis,:,:45000]])
    

all_df.shape